In [110]:
# settings
from IPython.display import display, Javascript

display(Javascript("""
MathJax.Hub.Config({
  "HTML-CSS": { scale: 85 },   // percentage: 100 is default
  SVG: { scale: 85 }
});
MathJax.Hub.Queue(["Rerender", MathJax.Hub]);
"""))


<IPython.core.display.Javascript object>

# Scattering Amplitude Reconstruction in Python

###### Giuseppe De Laurentis - University of Edinburgh

[PyHEP indico]("https://indico.cern.ch/event/1566263/timetable/#10-scattering-amplitude-recons")

## Motivation

Next-to-next-to-leading order predictions are essential to match the experimental precision

<img src="1911.00479.crosssection.png" style="max-width:150mm; margin-top:2mm; margin-bottom:0mm;">
<center> $\sigma_{pp\rightarrow \gamma\gamma\gamma}$ Chawdhry et al. <a href=https://arxiv.org/abs/1911.00479> arXiv:1911.00479 </a> </center>

without them we cannot tell new physics apart from a mismodelled SM background.

Partonic cross sections read:
$$ \small \hat{σ}_{n}=\frac{1}{2\hat{s}}\int d\Pi_{n-2}\;(2π)^4δ^4\big(\sum_{i=1}^n p_i\big)\;|\overline{\mathcal{A}(p_i,h_i,a_i,μ_F, μ_R)}|^2$$

Objective: compute the amplitude $ \mathcal{A} \approx \mathcal{A}^{\text{tree}} + \alpha_s \mathcal{A}^{(1-\text{loop})} + \alpha_s^2 \mathcal{A}^{(2-\text{loop})}$
1) At NNLO (~ two-loop Feynman diagrams) analytic computations are often unfeasible, especially at high multiplicities
2) Even if an analytic result is avaiable, they may be hard to simplify through analytic manipulations

Simpler results are not just a theorist delight, they are essential for phenomenology

<img src="quad-precision-no-rescue.png" style="max-width:150mm; margin-top:2mm; margin-bottom:0mm;">

<center> Evaluation of the two-loop corrections to pp -> Vjj obtained in 2021, Abreu et al.<a href=https://arxiv.org/abs/2110.07541> arXiv:2110.07541 </a> </center>

<img src="h2__g_g__Z_b_b.stability.png" style="max-width:150mm; margin-top:2mm; margin-bottom:0mm;">

<center> Evaluation of the new two-loop amplitude for pp -> Vjj, GDL et al.<a href=https://arxiv.org/abs/2503.1059> arXiv:2503.10595 </a> </center>

## What is analytic reconstruction?

A) Compute numerically to bypass all intermediate complexities <br>
B) Analytise the numerical evaluations to infer the exact analytic representation 

<br>

<center> <b> Analytic reconstruction is an alternative to analytic manipulations </b> </center>

<br>

A) it is needed both to obtain new NNLO corrections <br>
B) and to make available expressions efficient / usable

Efficiency is also a pressing need: the $\alpha_s$extraction from energy-energy correlators in $pp\rightarrow jjj$ by ATLAS [arXiv:2301.09351](https://arxiv.org/abs/2301.09351) used $10^8$CPUh

## Why Python over Mathematica?

Cons:
1) analytic manipulations are more awkward - but we will do mostly numerical work

Pros:
1) Name spaces: variables in Python are local by default, in Mathematica they are global
2) Object oriented: in Python we can attach methods and properties to objects, in Mathematica everything is a glorified list
3) Package manager (PyPI), in Mathematica pass files by email or attach to papers <br>
3b. Large community with many available packages
4) Continuous integration: pytest, flint, etc. - in Mathemtica, write a Python script that runs the test
5) Licenses: multi-processing / threading not limited by number of bought licences
6) Open source

Is performance an issue? Not really, outsource compute intensive operations to C++ / CUDA / Fortran / Rust (interfacing is simple)!

## A toy example

Work with spinor-helicity variables:
$$p_{i, \mu} \sigma_\mu^{\dot\alpha\alpha} = p^{\dot\alpha\alpha}_i = \tilde\lambda_i^{\dot\alpha}\lambda_i^{\alpha} = |i]\langle i | \; \text{ if } \;  p^2 = 0 $$
and Mandelstam invariants:
$$s_{ij\dots k} = (p_i + p_j + \dots + p_k)^2 $$

In [1]:
mandelstam_expression = "(1/(⟨14⟩^2⟨15⟩^2⟨23⟩^2))⟨12⟩^3⟨13⟩((4s23(-(s23s34+(s15-s34)s45)^3(s23s34+s45(s15+s34+s45))+s12^3(s15-s23)(s15^3s45+s23^2s34(-s23+s45)+s15^2s45(-s23+s45)+s15(s23^2s34-s23s45^2-s34s45^2))-s12^2(3s15^4s45^2+s15^3s45^2(-4s23-2s34+3s45)+s23s34^2(3s23^3-4s23^2s45+s45^3)+s15^2(-s23s45^2(s34+4s45)-s34s45^2(s34+5s45)+s23^2(s34^2+s45^2))+s15(-4s23^3s34^2+2s34^2s45^3+s23s34s45^2(s34+2s45)+s23^2s45(s34^2+s45^2)))+s12(3s15^4s45^3+s15^3s45^2(4s23s34-2s23s45-4s34s45+3s45^2)+s34^2(s23-s45)^2(3s23^2s34-s34s45^2+s23s45(s34+s45))-s15^2s45(s23^2s34(s34+s45)+s34s45^2(s34+7s45)+2s23s45(2s34^2-s34s45+s45^2))-s15s34(s23-s45)(2s23^2s34(s34-2s45)+s34s45^2(2s34+5s45)+s23s45(2s34^2+2s34s45+s45^2)))))/(3s12^3(s15-s23)s34(s12+s23-s45)s45(s15+s45)(-s12+s34+s45))+(4s23((s23s34+(s15-s34)s45)^2(s23s34+s45(s15+s34+s45))+s12^2(s23^2s34(s23-s45)+s15^3s45+s15^2s45(-s23+s45)-s15(s23^2s34+s23s45^2+s34s45^2))+s12(-2s15^3s45^2+s34^2(-2s23^3+2s23^2s45+s23s45^2-s45^3)+s15^2s45((s34-2s45)s45+s23(-s34+s45))+s15(s23^2s34(s34-s45)+s23s45^3+s34s45^2(s34+3s45))))(-tr5_1234))/(3s12^3(s15-s23)s34(s12+s23-s45)(s12-s34-s45)s45(s15+s45)))[31]"

def black_box_function(phase_space_point):  # may be a complicated expression we want to simplify, or a numerical routine
    return phase_space_point(mandelstam_expression)

In [2]:
from antares import Terms  # pip install antares-hep (Automated Numerical To Analytical Reconstruction Software)

In [3]:
# Goal: reconstruct the simplest possible form of this function
oTerms = Terms("""
+(8/3s23⟨24⟩[34])/(⟨15⟩⟨34⟩⟨45⟩⟨4|1+5|4])
+('12354', False, '+')
""")

### Flash overview of lips

In [4]:
from lips import Particles  # pip install lips (Lorentz invariant phase space)

In [5]:
# random (complex) phase space point with 5 massless legs, default is complex, 300 digits of precision
# uses mpmath for the arbitrary precision floating-point numbers (mpc and mpf)
oPs = Particles(5)

In [6]:
oPs[1].r_sp_d  # right-handed spinor index down

array([[mpc(real='1.06519515707852037181938050875665816944626450549501003019707928868798746156919715239314547852675960171700486281387719126976993813094246240128271678529047685153906956371844918881193706644420444381395050944543596222913763159298585875726262924292555038154943602044120743903491273135851182764855180556213111', imag='0.550130356046122653352469047659364994309992774608979736321256805355293715707992264220432994856733475486702054922377185128965835790285960993762331653101579554784671849022714565715782525946126568499169977483760396441114752591266061545999174636755337153920977065967441992918927045685687671584613452343955825')],
       [mpc(real='-0.10479000570805064385657327772177488659209214212951124762257313990559198519475549437691936835725998993538612942660635160268440833827006213101172908871444694987581824328536269395791915648640202301984055969163147944999886405611360263585515746752478393639389380483739785623104222277090122949532173374961024', imag='-0.163173072876150087137874

i.e. $\tilde\lambda_{1, \dot\alpha}$

In [7]:
oPs[1].l_sp_d  # left-handed spinor index down

array([[mpc(real='1.06519515707852037181938050875665816944626450549501003019707928868798746156919715239314547852675960171700486281387719126976993813094246240128271678529047685153906956371844918881193706644420444381395050944543596222913763159298585875726262924292555038154943602044120743903491273135851182764855180556213111', imag='0.550130356046122653352469047659364994309992774608979736321256805355293715707992264220432994856733475486702054922377185128965835790285960993762331653101579554784671849022714565715782525946126568499169977483760396441114752591266061545999174636755337153920977065967441992918927045685687671584613452343955825'),
        mpc(real='-0.944830204071188971358287474990524271728100084176299867460490819590874923727728469827041565840701930418971284939280395067056412021619520281450432683156160809758861221864603119627359402567276809052772155644597768338412445808389918549274694374864853609499800097582848537116886205911640787678869935211517782', imag='-0.062845489884750712017060

i.e. $\lambda_{1, \alpha}$

In [8]:
oPs[1].four_mom # p^\mu

array([mpc(real='0.460375692398478387541186540016682003934048802737625783809313235420523421265640713463679462711671794134888572620097022358139133633975041535678517360759226664495666619952100901067636814556229214658361002777491840772692315882541013766755555371745109979776788625685337448559059091999982705938371904419413683', imag='0.666374404516334030529369718660655849627767585454491572644603925264225668582837673868478249930653529757209921194915832346844420824349653828580179672822738286553557304073460526723972604208962301681105246976937451896741599635054848019346064995567148531846758606554230778176908579592873841464344051255712042'),
       mpc(real='-0.496855345911949685534591194968553459119496855345911949685534591194968553459119496855345911949685534591194968553459119496855345911949685534591194968553459119496855345911949685534591194968553459119496855345911949685534591194968553459119496855345911949685534591194968553459119496855345911949685534591194957', imag='-0.4090909090909090909090909

Implements getters and setters, e.g. updating the value of .four_mom will automatically update the spinors (and vice versa)

In [9]:
# I've implemented a custom ast (abstract syntactic tree) parser (this is hidden behind Particles.__call__)
oPs._parse(mandelstam_expression)

"(1/(oPs('⟨1|4⟩')**2*oPs('⟨1|5⟩')**2*oPs('⟨2|3⟩')**2))*oPs('⟨1|2⟩')**3*oPs('⟨1|3⟩')*((4*oPs('s_23')*(-(oPs('s_23')*oPs('s_34')+(oPs('s_15')-oPs('s_34'))*oPs('s_45'))**3*(oPs('s_23')*oPs('s_34')+oPs('s_45')*(oPs('s_15')+oPs('s_34')+oPs('s_45')))+oPs('s_12')**3*(oPs('s_15')-oPs('s_23'))*(oPs('s_15')**3*oPs('s_45')+oPs('s_23')**2*oPs('s_34')*(-oPs('s_23')+oPs('s_45'))+oPs('s_15')**2*oPs('s_45')*(-oPs('s_23')+oPs('s_45'))+oPs('s_15')*(oPs('s_23')**2*oPs('s_34')-oPs('s_23')*oPs('s_45')**2-oPs('s_34')*oPs('s_45')**2))-oPs('s_12')**2*(3*oPs('s_15')**4*oPs('s_45')**2+oPs('s_15')**3*oPs('s_45')**2*(-4*oPs('s_23')-2*oPs('s_34')+3*oPs('s_45'))+oPs('s_23')*oPs('s_34')**2*(3*oPs('s_23')**3-4*oPs('s_23')**2*oPs('s_45')+oPs('s_45')**3)+oPs('s_15')**2*(-oPs('s_23')*oPs('s_45')**2*(oPs('s_34')+4*oPs('s_45'))-oPs('s_34')*oPs('s_45')**2*(oPs('s_34')+5*oPs('s_45'))+oPs('s_23')**2*(oPs('s_34')**2+oPs('s_45')**2))+oPs('s_15')*(-4*oPs('s_23')**3*oPs('s_34')**2+2*oPs('s_34')**2*oPs('s_45')**3+oPs('s_23')*oPs(

In [10]:
complex(oTerms(oPs) - oPs(mandelstam_expression)) # evaluation of Terms object is optimized (including upcoming GPU support)

(-2.426485408108369e-300+1.2132427040541845e-300j)

In [11]:
expr = "+(8/3s23⟨24⟩[34])/(⟨15⟩⟨34⟩⟨45⟩⟨4|1+5|4])"
complex((oPs(expr) + oPs.image(('12354', False))(expr)) - oTerms(oPs))

(9.332636185032189e-302-1.8665272370064378e-301j)

In [12]:
# this generates a "real" phase space point (all outgoing convention, energy might be negative)
oPs = Particles(5, real_momenta=True, seed=0)

In [13]:
oPs[1].four_mom

array([mpc(real='0.687332462114479122995963240764965908335348316492734757090320627057009436362388736287994730768383742948855720784042793946320018929144633546574257505225786361677852508913182333102984702295913493771260722480915077025539504603705600363850362366641751820102176413143492357596828095988763161293061866584024905', imag='0.0'),
       mpc(real='-0.010256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256411', imag='0.0'),
       mpc(real='0.63636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636367',

### Main issue: numerical stability

In [14]:
from syngular import Field  # pip install syngular (interface and extension to Singular, https://www.singular.uni-kl.de/)
# syntax: Field("field name", characteristic, number of digits)

Simulate a long running computation where error accumulates by lowering the precision

In [15]:
oPs = Particles(5, field=Field("mpc", 0, 6), seed=0)

In [16]:
abs(oTerms(oPs) - black_box_function(oPs))

mpf('2.38776875e-6')

In [17]:
oPs = Particles(5, field=Field("mpc", 0, 6), seed=2)

In [18]:
abs(oTerms(oPs) - black_box_function(oPs))  # lost precision! :'(

mpf('0.428962946')

In [19]:
black_box_function(oPs)

mpc(real='-1.13071203', imag='25.7996826')

In [20]:
oTerms(oPs)

mpc(real='-0.711602092', imag='25.8910942')

In [21]:
oPs = Particles(5, field=Field("mpc", 0, 16), seed=2)

In [22]:
black_box_function(oPs)

mpc(real='-0.7116312918164890561', imag='25.89107035069941443')

In [23]:
oTerms(oPs)

mpc(real='-0.7116312918068779803', imag='25.89107035076286678')

## finite fields

$$ 
\displaystyle a \in \mathbb{F}_p : a \in \{0, \dots, p -1\} \; \text{ with } \; \{+, -, \times, \div\}
$$

In [24]:
from fractions import Fraction as Q
from pyadic import ModP

In [25]:
ModP(Q(1, 2), 11)

6 % 11

or

In [26]:
from syngular import Field

In [27]:
Fp = Field("finite field", 2 ** 31 - 1, 1)
Fp

Field('finite field', 2147483647, 1)

In [28]:
Fp.random()

1726693670 % 2147483647

what goes wrong if p is not prime?

In [103]:
# ModP(Q(1, 2), 6)  # this raises ZeroDivisionError

the inverse cannot be computed for any number not co-prime with the chosen modulus

### Phase space in a finite field

In [30]:
oPs = Particles(5, field=Field("finite field", 2 ** 31 - 1, 1), seed=0)

In [31]:
black_box_function(oPs)

497973027 % 2147483647

In [32]:
oTerms(oPs)

497973027 % 2147483647

### The trivial absoulute value

$$ |a|_0 = 0 \; \text{if} \; a = 0 \; \text{else} \; 1$$

In [33]:
abs(ModP(Q(2), 5))

1

In [34]:
abs(ModP(Q(0), 5))

0

### Limitation

a) can make things zero, but not small <br>
b) cannot take limits <br>
c) calculus is undefined

In [104]:
# Fp.epsilon()  # This raises ValueError (no finite field infinitesimal exists)

### [Ostrowski theorem](https://en.wikipedia.org/wiki/Ostrowski%27s_theorem)

There exist 3 possible absolute values on the rationals: <br>
     - the trivial absolute value $|\cdot|_0$ <br>
     - the real absolute value $|\cdot|_\infty$ <br>
     - the p-adic absolute value $|\cdot|_p$

## p-adic numbers

In [36]:
from pyadic import PAdic

In [37]:
ModP(13, 11)

2 % 11

In [38]:
PAdic(13, 11, 3)

2 + 1*11 + O(11^3)

In [39]:
ModP(Q(1, 2), 11)

6 % 11

In [40]:
PAdic(Q(1, 2), 11, 3)

6 + 5*11 + 5*11^2 + O(11^3)

In [41]:
# we can divide by p

In [42]:
PAdic(Q(1, 11), 11, 3)

1*11^-1 + O(11^2)

Absolute value goes in descrete steps of powers of $p$ (negative powers are "large", think of $p$ as $\epsilon \ll 1$)

In [115]:
abs(PAdic(Q(1, 11), 11, 3))

O(11^-1)

In [116]:
abs(PAdic(Q(11), 11, 3))

O(11)

In [117]:
abs(PAdic(Q(1, 11), 11, 3)) > abs(PAdic(Q(11), 11, 3))

True

In [119]:
abs(PAdic(Q(12), 11, 3)) == abs(PAdic(Q(13), 11, 3))

True

In [105]:
# ModP(Q(1, 11), 11)  # this raises ZeroDivisionError: cannot divide by the prime

In [44]:
Qp = Field("padic", 2 ** 31 - 1, 11)
Qp

Field('padic', 2147483647, 11)

In [45]:
Qp.random()

1040910454 + 2111760762*2147483647 + 1390445792*2147483647^2 + 151754545*2147483647^3 + 900812049*2147483647^4 + 867972296*2147483647^5 + 276429464*2147483647^6 + 1192149140*2147483647^7 + 1772713530*2147483647^8 + 439161975*2147483647^9 + 1071994858*2147483647^10 + O(2147483647^11)

In [46]:
Qp.epsilon()  # we can do calculs! take limits, compute derivatives, even integrate

2147483647

### Improve Stability

The usual triangle inequality
$$\displaystyle |d(x,z)| \leq |d(x,y)| + |d(y,z)| $$
is strengthened to
$${\displaystyle d(x,z)\leq \max \left\{d(x,y),d(y,z)\right\}}$$

<b> Can never add two numbers and get a larger one than those you started with </b>

### p-adic phase space

In [47]:
oPs = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)

In [48]:
black_box_function(oPs)

1222473774 + 796252591*2147483647 + 1161721493*2147483647^2 + O(2147483647^3)

In [49]:
oTerms(oPs)

1222473774 + 796252591*2147483647 + 1161721493*2147483647^2 + O(2147483647^3)

## Interpolation algorithms

Peraro [arXiv:1608.01902](https://arxiv.org/abs/1608.01902)

In [57]:
from pyadic.interpolation import Newton_polynomial_interpolation, Thiele_rational_interpolation

In [60]:
def univariate_polynomial(tval):
    return (tval ** 10 + tval - 1)

In [61]:
Newton_polynomial_interpolation(univariate_polynomial, 2 ** 31 - 1, verbose=True)

Finished after 13 samples: [1569849180 % 2147483647, 1078182456 % 2147483647, 1345237137 % 2147483647, 1566393810 % 2147483647, 815006192 % 2147483647, 319562461 % 2147483647, 1293390443 % 2147483647, 677331714 % 2147483647, 380413367 % 2147483647, 1203676396 % 2147483647, 1 % 2147483647, 0 % 2147483647, 0 % 2147483647]. 

t**10 + t - 1

In [58]:
def univariate_rational_func(tval):
    return (tval ** 20 + tval - 1) / (5 - tval)

In [59]:
Thiele_rational_interpolation(univariate_rational_func, 2 ** 31 - 1, verbose=True)

Finished after 40 samples, [584414832 % 2147483647, 970812621 % 2147483647, 1534536413 % 2147483647, 954387064 % 2147483647, 359570102 % 2147483647, 1570313059 % 2147483647, 109498696 % 2147483647, 2123742756 % 2147483647, 1386017183 % 2147483647, 1535674705 % 2147483647, 1521506693 % 2147483647, 1184513925 % 2147483647, 958941582 % 2147483647, 1184542341 % 2147483647, 1008474606 % 2147483647, 1717109627 % 2147483647, 399620187 % 2147483647, 508518201 % 2147483647, 466795340 % 2147483647, 1573390819 % 2147483647, 803901946 % 2147483647, 437215979 % 2147483647, 454478843 % 2147483647, 208954972 % 2147483647, 1633357754 % 2147483647, 1831680191 % 2147483647, 1914540195 % 2147483647, 516669207 % 2147483647, 1330111113 % 2147483647, 1923328681 % 2147483647, 1119820556 % 2147483647, 917973577 % 2147483647, 627695372 % 2147483647, 1365942856 % 2147483647, 789941148 % 2147483647, 1024635069 % 2147483647, 1542141675 % 2147483647, 1565654157 % 2147483647, 376073685 % 2147483647, 507260310 % 214

(t**20 + t - 1)/(5 - t)

### Why Reconstruction $\supset$ Interpolation

As far as I am aware, interpolation algorithms work only for independent sets of variables.

We have variables subject to constraints (e.g. momentum conservation, Schouten identities, etc.)

## Least Common Denominator (LCD)

In [77]:
field = Field("finite field", 2 ** 31 - 1, 1)
seed = 0 
oSliceFp = Particles(5, field=field, seed=seed)
oSliceFp.univariate_slice(algorithm='covariant', seed=seed)

In [78]:
import sympy

In [79]:
# momentum conservation is satisfied for all values of the univariate parameter t
sympy.poly(sympy.simplify(oSliceFp.total_mom)[1, 0], modulus=2** 31- 1)

Poly(0, t, modulus=2147483647)

In [80]:
from antares import settings
from antares.core.numerical_methods import num_func

from lips import Invariants

In [81]:
settings.invariants = Invariants(5).full
print(settings.invariants)  # list of guesses for what the possible singularities of the amplitude can be

['⟨1|2⟩', '[1|2]', '⟨1|3⟩', '[1|3]', '⟨1|4⟩', '[1|4]', '⟨1|5⟩', '[1|5]', '⟨2|3⟩', '[2|3]', '⟨2|4⟩', '[2|4]', '⟨2|5⟩', '[2|5]', '⟨3|4⟩', '[3|4]', '⟨3|5⟩', '[3|5]', '⟨4|5⟩', '[4|5]', '⟨1|(2+3)|1]', '⟨1|(2+5)|1]', '⟨2|(1+3)|2]', '⟨2|(1+5)|2]', '⟨3|(1+2)|3]', '⟨3|(1+5)|3]', '⟨4|(1+2)|4]', '⟨4|(1+5)|4]', '⟨5|(1+2)|5]', '⟨5|(1+4)|5]', '⟨1|(2+3)|(2+5)|1⟩', '[1|(2+3)|(2+5)|1]', '⟨2|(1+3)|(1+5)|2⟩', '[2|(1+3)|(1+5)|2]', '⟨5|(1+2)|(1+4)|5⟩', '[5|(1+2)|(1+4)|5]', 'tr5_1234']


In [82]:
black_box_function.multiplicity = 5
oF = num_func(black_box_function)

In [83]:
x = sympy.poly(sympy.simplify(oSliceFp("⟨1|4⟩")), modulus=2** 31- 1)
x, x.factor_list()

(Poly(707309386*t + 976617537, t, modulus=2147483647),
 (707309386, [(Poly(t + 525474835, t, modulus=2147483647), 1)]))

In [84]:
oF.get_lcd(oSliceFp, verbose=True)

Finished after 19 samples, [162860843 % 2147483647, 1885044667 % 2147483647, 1411810336 % 2147483647, 1402519729 % 2147483647, 468240741 % 2147483647, 1197585802 % 2147483647, 1947316210 % 2147483647, 291407823 % 2147483647, 450073212 % 2147483647, 882656524 % 2147483647, 1372881572 % 2147483647, 934808415 % 2147483647, 1385619189 % 2147483647, 1938318280 % 2147483647, 1622103845 % 2147483647, 869915779 % 2147483647, 1364220815 % 2147483647, 64322534 % 2147483647, 552291472 % 2147483647]. 
 (-514009628*t**8 + 776677322*t**7 + 991846540*t**6 - 75374069*t**5 + 101225610*t**4 + 36666516*t**3 - 728369354*t**2 + 534567587*t - 70010385)/(t**9 - 49350670*t**8 + 1034569225*t**7 + 606286545*t**6 + 418505031*t**5 - 958181787*t**4 - 882182274*t**3 - 369000092*t**2 - 291896180*t - 698421050)
Polynomial defaultdict(<class 'int'>, {t - 505232661: 1, t - 179967131: 1, t + 594877252: 1, t + 969664725: 1, t**4 + 449954863*t**3 + 382707815*t**2 - 580761694*t - 422746403: 1})
Matched 2 / 8: {'⟨2|3⟩': 1, 

Terms("""+(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])""")

In [85]:
oTerms.get_lcd(oSliceFp)

Terms("""+(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])""")

<img src="variety_slice_v2-transparent.png" style="max-width:100mm; margin-top:2mm; margin-bottom:0mm;">
The blue line is analogous to the univariate slice, but our space has dimension $4n-4=20-4=16$

The planes $\mathcal{D}_i = 0$ are analogous to our denominator factors, but they have dimension $16-1 = 15$

In [99]:
# Univariate interpolation for the LCD is equivalent to checking limits:
field = Field("padic", 2 ** 31 - 1, 5)
seed = 0 
oPs = Particles(5, field=field, seed=seed)
oPs._singular_variety(("⟨1|4⟩", ), (1, ))

In [100]:
oF(oPs)  # simple pole

667847873*2147483647^-1 + 53022262 + 1129123795*2147483647 + 632358862*2147483647^2 + O(2147483647^3)

In [101]:
oPs._singular_variety(("⟨2|3⟩", ), (1, ))

In [102]:
oF(oPs)  # degree 1 zero

321959245*2147483647 + 1080775670*2147483647^2 + O(2147483647^3)

Problem: in LCD form the expressions are way to complicated

For instance, for Vjj (at leading color), the largest numerator in LCD form had mass dimension 114, its ansatz size has approx 25M free parameters.

Solution: do partial fraction decompositions <br>
To do this in a multivariate setting we need some algebraic geometry

## Computational Algebraic Geometry

In [131]:
from lips.algebraic_geometry.covariant_ideal import LipsIdeal

In [132]:
oPs = Particles(5)
oPs.make_analytical_d()

In [133]:
oPs["|1⟩"], oPs["[1|"]

(array([[a1],
        [b1]], dtype=object),
 array([[c1, d1]], dtype=object))

In [134]:
LipsIdeal.__bases__

(syngular.ideal.Ideal,)

### Geometry of singular phase space

In [135]:
oPs = Particles(5, field=Field("padic", 2 ** 31 - 1, 5))

In [136]:
J = LipsIdeal(5, ("⟨4|1+5|4]", "⟨5|1+4|5]", ))
J

Ideal over Ring
    0, (a1, b1, c1, d1, a2, b2, c2, d2, a3, b3, c3, d3, a4, b4, c4, d4, a5, b5, c5, d5), dp
generated by
    4*a1*b4*c1*d4 - 4*a1*b4*c4*d1 - 4*a4*b1*c1*d4 + 4*a4*b1*c4*d1 + 4*a4*b5*c4*d5 - 4*a4*b5*c5*d4 - 4*a5*b4*c4*d5 + 4*a5*b4*c5*d4,4*a1*b5*c1*d5 - 4*a1*b5*c5*d1 + 4*a4*b5*c4*d5 - 4*a4*b5*c5*d4 - 4*a5*b1*c1*d5 + 4*a5*b1*c5*d1 - 4*a5*b4*c4*d5 + 4*a5*b4*c5*d4,b1*d1 + b2*d2 + b3*d3 + b4*d4 + b5*d5,-a1*d1 - a2*d2 - a3*d3 - a4*d4 - a5*d5,-b1*c1 - b2*c2 - b3*c3 - b4*c4 - b5*c5,a1*c1 + a2*c2 + a3*c3 + a4*c4 + a5*c5

The following line checks whether this ideal is prime (it isn't)

In [137]:
J.test_primality(verbose=True)

gathering f-poly factors: @ 37/38 of which 0 timedout                                   
easiest projection is 208: (1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1) of degree 1
The ideal is radical: True
 smallest f poly factors (33): complexity 856                              
Original ideal has codim 6
 at factor 0: 1.                                       
 at factor 1: a1.                                       
 at factor 2: a3.                                       
 at factor 3: d3.                                       
 at factor 4: d2.                                       
 at factor 5: d5.                                       
 at factor 6: c2.                                       
 at factor 7: b5.                                       
 at factor 8: d4.                                       
 at factor 9: a4.                                       
 at factor 10: b4.                                       
 at factor 11: b1.                                    

False

We need the following 3 prime ideals

In [138]:
K = LipsIdeal(5, ("⟨14⟩", "⟨15⟩", "⟨45⟩", "[23]"))
L = LipsIdeal(5, ("⟨12⟩", "⟨13⟩", "⟨14⟩", "⟨15⟩", "⟨23⟩", "⟨24⟩", "⟨25⟩", "⟨34⟩", "⟨35⟩", "⟨45⟩"))
M = LipsIdeal(5, ("⟨4|1+5|4]", "⟨5|1+4|5]", "|1]⟨14⟩⟨15⟩+|4]⟨14⟩⟨45⟩-|5]⟨45⟩⟨15⟩", "|1⟩[14][15]+|4⟩[14][45]-|5⟩[45][15]"))

The following verifies they are indeed prime

In [139]:
assert K.test_primality() and L.test_primality() and M.test_primality()

\& operator means intersection ($\cap$), like for sets. The following checks that the ideal J is indeed an intersection of 5 prime ideals.

In [140]:
assert K & K("12345", True) & L & L("12345", True) & M == J

In other words, the variety (= hyper-surface) V(J) is the union of V(K), V(K-bar), V(L), V(L-bar) and V(M)

### Phase-space points on irreducible varieties

We can now use this to find if a partial fraction decomposition is possible, using numerics only. <br>
Normally, we have access to a "black box function" for the rational expression, and the common denominator.

In [141]:
black_box_function = "(8/3s23⟨24⟩[34])/(⟨15⟩⟨34⟩⟨45⟩⟨4|1+5|4])+(8/3s23⟨25⟩[35])/(⟨14⟩⟨35⟩⟨54⟩⟨5|1+4|5])"
common_denominator = "(⟨14⟩⟨15⟩⟨34⟩⟨35⟩⟨45⟩⟨4|1+5|4]⟨5|1+4|5])"

In [142]:
oPsK = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)
oPsK._singular_variety(("⟨4|1+5|4]", "⟨5|1+4|5]"), (1, 1), generators=K.generators)

In [143]:
oPsK(black_box_function) * oPsK(common_denominator)  # rational * denominator is a proxy for the numerator polynomial

1268828065*2147483647^3 + 440056888*2147483647^4 + O(2147483647^5)

In [144]:
oPsKb = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)
oPsKb._singular_variety(("⟨4|1+5|4]", "⟨5|1+4|5]"), (1, 1), generators=K("12345", True).generators)

In [145]:
oPsKb(black_box_function) * oPsKb(common_denominator)

459307737*2147483647^2 + 956093676*2147483647^3 + O(2147483647^4)

In [146]:
oPsL = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)
oPsL._singular_variety(("⟨4|1+5|4]", "⟨5|1+4|5]"), (1, 1), generators=L.generators)

In [147]:
oPsL(black_box_function) * oPsL(common_denominator)

1158980278*2147483647^5 + 1574552815*2147483647^6 + O(2147483647^7)

In [148]:
oPsLb = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)
oPsLb._singular_variety(("⟨4|1+5|4]", "⟨5|1+4|5]"), (1, 1), generators=L("12345", True).generators)

In [149]:
oPsL(black_box_function) * oPsL(common_denominator)

1158980278*2147483647^5 + 1574552815*2147483647^6 + O(2147483647^7)

In [150]:
oPsM = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)
oPsM._singular_variety(("⟨4|1+5|4]", "⟨5|1+4|5]"), (1, 1), generators=M.generators)

In [151]:
oPsM(black_box_function) * oPsM(common_denominator)

275934960*2147483647 + 939134786*2147483647^2 + O(2147483647^3)

## Outlook

In [54]:
from antares.core.unknown import Unknown
from antares.numerical_to_analytical import numerical_to_analytical

In [58]:
oF.do_single_collinear_limits()
oF.do_double_collinear_limits()

The partial result is:                                                                                             
(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])

Mass dimension & phase weights: -1.0, [-1, 1, -2, -2, -2] → 6.0, [1, 1, 0, 1, 1]
Cleaning pair scalings results from known numerator information.                                                             
Finished calculating pair scalings. They are:                         
[⟨1|4⟩, ⟨1|5⟩]:             3.0, 31 → 7
[⟨1|4⟩, ⟨3|4⟩]:             3.0, 26 → 7
[⟨1|4⟩, ⟨3|5⟩]:             2.0, 2  → 2
[⟨1|4⟩, ⟨4|5⟩]:             3.0, 31 → 7
[⟨1|4⟩, ⟨4|(1+5)|4]]:       1.0, 3  → 2
[⟨1|4⟩, ⟨5|(1+4)|5]]:       2.0, 4  → 2
[⟨1|5⟩, ⟨3|4⟩]:             2.0, 4  → 2
[⟨1|5⟩, ⟨3|5⟩]:             3.0, 26 → 7
[⟨1|5⟩, ⟨4|5⟩]:             3.0, 31 → 7
[⟨1|5⟩, ⟨4|(1+5)|4]]:       2.0, 3  → 2
[⟨1|5⟩, ⟨5|(1+4)|5]]:       1.0, 3  → 2
[⟨3|4⟩, ⟨3|5⟩]:             3.0, 31 → 7
[⟨3|4⟩, ⟨4|5⟩]:             3.0, 31 → 7
[⟨3|4⟩, ⟨4|(1+5)|4]]:

In [59]:
oU = Unknown(oF)

In [60]:
numerical_to_analytical(oU)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
NoneToAnalytical called at depth 0.

The partial result is:                                                                                             
(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])

Mass dimension & phase weights: -1.0, [-1, 1, -2, -2, -2] → 6.0, [1, 1, 0, 1, 1]
Cleaning pair scalings results from known numerator information.                                                             
Finished calculating pair scalings. They are:                         
[⟨1|4⟩, ⟨1|5⟩]:             3.0 (3.0), 31 → 7
[⟨1|4⟩, ⟨3|4⟩]:             3.0 (3.0), 26 → 7
[⟨1|4⟩, ⟨3|5⟩]:             2.0 (2.0), 2  → 2
[⟨1|4⟩, ⟨4|5⟩]:             3.0 (3.0), 31 → 7
[⟨1|4⟩, ⟨4|(1+5)|4]]:       1.0 (1.0), 3  → 2
[⟨1|4⟩, ⟨5|(1+4)|5]]:       2.0 (2.0), 4  → 2
[⟨1|5⟩, ⟨3|4⟩]:             2.0 (2.0), 4  → 2
[⟨1|5⟩, ⟨3|5⟩]:             3.0 (3.0), 26 → 7
[⟨1|5⟩, ⟨4|5⟩]:             3.0 (3.0), 31 → 7
[⟨1|

[Terms("""+[2|3]⟨2|3⟩(8/3⟨2|5⟩⟨1|4⟩⟨2|4⟩⟨3|5⟩[2|3][4|5]-8/3⟨4|5⟩⟨2|5⟩⟨2|4⟩[3|5]⟨1|3⟩[2|4]-16/3⟨4|5⟩⟨2|4⟩⟨3|5⟩[3|5]⟨1|3⟩[3|4]+8/3⟨4|5⟩⟨1|2⟩⟨3|5⟩⟨3|4⟩[3|5][3|4]+8/3⟨4|5⟩²[3|5]⟨1|3⟩⟨2|3⟩[3|4])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])""")]

## GitHub self hosted runner (for GPU access)